In [1]:
!pip install mlflow torchvision>=0.9.1 torch==1.9.0 pytorch-lightning==1.4.0

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/mlflow/mlflow.git

fatal: destination path 'mlflow' already exists and is not an empty directory.


In [4]:
%cd mlflow/examples/pytorch/MNIST

/tmp/mlflow/examples/pytorch/MNIST


In [5]:
!mlflow run . --no-conda

2021/09/03 17:12:06 INFO mlflow.projects.utils: === Created directory /tmp/tmp6v5nz5hx for downloading remote URIs passed to arguments of type 'path' ===
2021/09/03 17:12:06 INFO mlflow.projects.backend.local: === Running command 'python mnist_autolog_example.py \
  --max_epochs 5 \
  --gpus 0 \
  --accelerator None \
  --batch_size 64 \
  --num_workers 3 \
  --lr 0.001 \
  --es_patience 3 \
  --es_mode min \
  --es_verbose True \
  --es_monitor val_loss
' in run with ID '605a74852ebe42f7acdfba013cfb9307' === 
/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. 

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
Testing: 0it [00:00, ?it/s]/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
Testing:  92%|████████████████████████████▋  | 145/157 [00:00<00:00, 217.71it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_acc': 0.9748208522796631}
--------------------------------------------------------------------

In [6]:
!tree -L 1 mlruns/0

mlruns/0
├── 4bb45ba9395b48feb5a76ded43e4f37d
├── 605a74852ebe42f7acdfba013cfb9307
├── 860c30a3e4b7436188128a5715ebbe80
└── meta.yaml

3 directories, 1 file


In [26]:
from torchvision import datasets

# load the training data
mnist_test = datasets.MNIST('/tmp/data', train=False, download=True)
mnist_test = list(mnist_test)[1]
img, category = mnist_test
display(img)
print(category)

2


In [28]:
import numpy as np
img_np = np.asarray(img).reshape((1, 28*28)).astype(np.float32)

In [20]:
import os
ARTIFACTS_FOLDER = os.path.join(
    os.getcwd(),
    "mlruns/0",
    os.listdir("mlruns/0")[0],
    "artifacts",
    "model"
)
print(ARTIFACTS_FOLDER)

/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/605a74852ebe42f7acdfba013cfb9307/artifacts/model


In [21]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model

mlflow_tag = "mlflow"

pytorch_mnist_model = Model(
    name="test-pytorch-mnist",
    platform=ModelFramework.MLFlow,
    local_folder=ARTIFACTS_FOLDER,
    uri="s3://tempo/basic/mnist",
    description="A pytorch MNIST model",
)



Insights Manager not initialised as empty URL provided.


In [22]:
from tempo.serve.loader import save
save(pytorch_mnist_model)

Solving environment: ...working... done

_openmp_mutex-4.5    | 22 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... Ran pip subprocess with arguments:
['/home/sa/miniconda3/envs/tempo-9a2089d9-aed4-45b7-a932-9f9bd905e625/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.joi7ad7o.requirements.txt']
Pip subprocess output:
  Using cached mlflow-1.20.1-py3-none-any.whl (14.6 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached cffi-1.14.6-cp37-cp37m-manylinux1_x86_64.whl (402 kB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached ipython-7.26.0-py3-none-any.whl (786 kB)
  Using cached pyDeprecate-0.3.1-py3-none-any.whl (10 kB)
  Using cached pytorch_lightning-1.4.0-py3

Packing environment at '/home/sa/miniconda3/envs/tempo-9a2089d9-aed4-45b7-a932-9f9bd905e625' to '/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/605a74852ebe42f7acdfba013cfb9307/artifacts/model/environment.tar.gz'
[########################################] | 100% Completed |  1min 23.1s

## Package Plan ##

  environment location: /home/sa/miniconda3/envs/tempo-9a2089d9-aed4-45b7-a932-9f9bd905e625


The following packages will be REMOVED:

  _libgcc_mutex-0.1-conda_forge
  _openmp_mutex-4.5-1_gnu
  ca-certificates-2021.5.30-ha878542_0
  ld_impl_linux-64-2.36.1-hea4e1c9_2
  libffi-3.3-h58526e2_2
  libgcc-ng-11.1.0-hc902ee8_8
  libgomp-11.1.0-hc902ee8_8
  libstdcxx-ng-11.1.0-h56837e0_8
  ncurses-6.2-h58526e2_4
  openssl-1.1.1l-h7f98852_0
  pip-21.2.4-pyhd8ed1ab_0
  python-3.7.9-hffdb5ce_100_cpython
  python_abi-3.7-2_cp37m
  readline-8.1-h46c0cb4_0
  setuptools-57.4.0-py37h89c1867_0
  sqlite-3.36.0-h9cd32fc_0
  tk-8.6.11-h27826a3_1
  wheel-0.37.0-pyhd8ed1ab_1
  xz-5.2.5-h516909a_1
  zlib-1.2


Remove all packages in environment /home/sa/miniconda3/envs/tempo-9a2089d9-aed4-45b7-a932-9f9bd905e625:



In [23]:
from tempo import deploy_local
local_deployed_model = deploy_local(pytorch_mnist_model)

In [29]:
local_prediction = local_deployed_model.predict(img_np)
print(np.nonzero(local_prediction.flatten() == 0))

(array([2]),)


In [30]:
local_deployed_model.undeploy()

In [31]:
%cd -0

/home/sa/code/tempo/docs/examples/mlflow_e2e


In [32]:
!kubectl apply -f k8s/rbac -n seldon

secret/minio-secret configured
serviceaccount/tempo-pipeline unchanged
role.rbac.authorization.k8s.io/tempo-pipeline unchanged
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding unchanged


In [33]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [34]:
from tempo.serve.loader import upload
upload(pytorch_mnist_model)

In [36]:
from tempo.serve.metadata import SeldonCoreOptions
runtime_options = SeldonCoreOptions(**{
        "remote_options": {
            "namespace": "seldon",
            "authSecretName": "minio-secret"
        }
    })

In [37]:
from tempo import deploy_remote
remote_deployed_model = deploy_remote(pytorch_mnist_model, options=runtime_options)

In [38]:
remote_prediction = remote_deployed_model.predict(img_np)
print(np.nonzero(remote_prediction.flatten() == 0))

(array([2]),)


In [39]:
remote_deployed_model.undeploy()